<a href="https://colab.research.google.com/github/mucahitozkaya/app/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.utils import to_categorical


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation,Dropout, BatchNormalization,MaxPool1D,LayerNormalization
from tensorflow.keras import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC  # Import Support Vector Classifier (SVC)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#Read datas

In [19]:
train_final = pd.read_parquet("https://drive.google.com/uc?id=1ZeXb2EBMkP-nzULonq_hDnIxF5u9PwKp")
test_final = pd.read_parquet("https://drive.google.com/uc?id=1Blepx7rAAUXFLLNIL8gCI2g1IcNoMfTk")
submission_sample = pd.read_parquet("https://drive.google.com/uc?id=1KCcrx84tmG5biO0MjLmnFhvCMSC-U6ge")

In [ ]:
df = train_final
df

In [4]:
label_encoder = LabelEncoder()
df['target_encoded'] = label_encoder.fit_transform(df['target'])
df['target_encoded'].nunique()

112

In [ ]:
features_corr = df.loc[:, 'feature_0':'feature_24']
features_corr['target_encoded'] = df['target_encoded']
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [ ]:
features_corr = df.loc[:, 'feature_25':'target_encoded']
features_corr = features_corr.drop(columns=['target'])
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [22]:
df['target_cleaned'] = df['target'].apply(lambda x: ','.join([value.strip() for value in x.split(',')]))

df[['target1', 'target2','target3']] = df['target_cleaned'].str.split(',', expand=True)

label_encoder = LabelEncoder()
df['target1'] = label_encoder.fit_transform(df['target1'])
df['target2'] = label_encoder.fit_transform(df['target2'])
df['target3'] = label_encoder.fit_transform(df['target3'])


,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_45,feature_46,feature_47,feature_48,feature_49,target,target_cleaned,target1,target2,target3
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,1.658986,-1.559406,-2.161336,30,58,"menu2, menu4, menu5","menu2,menu4,menu5",menu2,menu4,menu5
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,2.775513,-0.318980,-4.291473,21,45,"menu7, menu8, menu4","menu7,menu8,menu4",menu7,menu8,menu4
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,1.293131,-2.230909,-2.383524,19,61,"menu2, menu8, menu4","menu2,menu8,menu4",menu2,menu8,menu4
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,1.029710,-1.142185,-4.466191,2,41,"menu6, menu2, menu1","menu6,menu2,menu1",menu6,menu2,menu1
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,-0.061626,-1.462175,-2.371206,23,85,"menu6, menu2, menu8","menu6,menu2,menu8",menu6,menu2,menu8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,0.761354,1.073711,-3.560019,29,68,"menu2, menu8, menu4","menu2,menu8,menu4",menu2,menu8,menu4
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,-0.893553,-2.061401,-1.908158,25,43,"menu9, menu2, menu5","menu9,menu2,menu5",menu9,menu2,menu5
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,-0.495204,-0.955097,-3.843092,4,25,"menu6, menu2, menu4","menu6,menu2,menu4",menu6,menu2,menu4
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,-0.100755,-1.056479,-1.452037,25,60,"menu6, menu8, menu4","menu6,menu8,menu4",menu6,menu8,menu4


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94049 entries, 0 to 94048
Data columns (total 68 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              94049 non-null  object 
 1   month           94049 non-null  int64  
 2   n_seconds_1     94049 non-null  float64
 3   n_seconds_2     94049 non-null  float64
 4   n_seconds_3     94049 non-null  float64
 5   carrier         94049 non-null  object 
 6   devicebrand     94049 non-null  object 
 7   feature_0       94049 non-null  float64
 8   feature_1       94049 non-null  float64
 9   feature_2       94049 non-null  float64
 10  feature_3       94049 non-null  float64
 11  feature_4       94049 non-null  float64
 12  feature_5       94049 non-null  float64
 13  feature_6       94049 non-null  float64
 14  feature_7       94049 non-null  float64
 15  feature_8       94049 non-null  float64
 16  feature_9       94049 non-null  float64
 17  feature_10      94049 non-null 

#Call Back

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime

# eğitim sırasında en iyi model ağırlıklarını kayıt edelim ve
# eğitim performansını TensorBoard ile izleyelim

filepath="weights-improvement-{epoch:02d}-{val_accuracy:04f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
logdir="/content/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
callbacks_list = [checkpoint,tensorboard_callback]


#Preprocess

In [15]:

# Katogorik verilerin one hot encoding yapılarak numerik hale getirilmesi
object_list = []
for column in df.columns:
     if df[column].dtype == 'object':
        object_list.append(column)
try:
  object_list.remove('target')
  object_list.remove('target_cleaned')
  object_list.remove('id')
  df = df.drop(columns=['id','target','target_cleaned'])
except:
  pass

df = pd.get_dummies(df, columns=object_list)

df

,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,devicebrand_iBRIT,devicebrand_lge,devicebrand_meizu,devicebrand_motorola,devicebrand_nubia,devicebrand_realme,devicebrand_reeder,devicebrand_samsung,devicebrand_vivo,devicebrand_xiaomi
0,10,5245.571,981.182,205.948,-1.197737,1.113360,-1.123334,-0.263580,2.161242,2.651375,...,0,0,0,0,0,0,0,0,0,0
1,10,5184.876,557.650,487.587,-2.336352,2.567766,-0.494908,0.949101,3.567557,3.357848,...,0,0,0,0,0,0,0,1,0,0
2,10,3835.618,3275.128,43.806,-2.561455,2.061736,-0.184511,1.062306,4.197788,1.551181,...,0,0,0,0,0,0,0,0,0,0
3,10,3532.544,154.509,64.724,-2.529918,3.358050,-0.851366,1.643876,2.849205,3.887427,...,0,0,0,0,0,0,0,1,0,0
4,10,3344.192,787.896,715.115,-2.922361,2.096124,0.060796,-1.487557,3.224788,2.091947,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,12,44.397,43.425,41.678,-1.531534,2.596604,0.340233,-1.149720,3.308406,2.995904,...,0,0,0,0,0,0,0,0,0,0
94045,12,44.331,43.977,40.620,-1.268987,2.300487,0.231711,0.741582,2.365813,2.031927,...,0,0,0,0,0,0,0,1,0,0
94046,12,44.142,43.591,41.736,-1.950039,2.805681,0.438200,2.976427,4.001829,3.051181,...,0,0,0,0,0,0,0,0,0,0
94047,12,43.963,43.350,40.862,-2.389140,2.358281,0.683524,0.234449,4.070453,1.709853,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X,y = df.drop('target_encoded', axis=1), to_categorical(df['target_encoded'])
# Veri standartlaştırma
scaler = StandardScaler()
data_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(data_scaled,y, test_size=0.1, random_state=42)




In [15]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(80, activation="relu", name="layer1"),
        layers.Dense(63, activation="relu", name="layer2"),
         LayerNormalization(axis=1),
        layers.Dense(30, activation="relu", name="layer3"),
        layers.Dense(30, activation="relu", name="layer4"),
        Dropout(0.4),
        layers.Dense(60, activation="relu", name="layer5"),
        Dropout(0.4),
         LayerNormalization(axis=1),
        layers.Dense(80, activation="relu", name="layer6"),
        layers.Dense(112, name="layer7"),
    ]
)

# Modeli Derleme
model.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

history = model.fit(x=X_train,y=y_train,
                        epochs=150,
                        validation_data = (X_test,y_test),
                       callbacks=callbacks_list
                        )

Epoch 1/150
2644/2646 [============================>.] - ETA: 0s - loss: 8.2418 - accuracy: 0.0491
Epoch 1: val_accuracy did not improve from 0.13110
2646/2646 [==============================] - 20s 6ms/step - loss: 8.2409 - accuracy: 0.0491 - val_loss: 6.6047 - val_accuracy: 0.0737
Epoch 2/150
2641/2646 [============================>.] - ETA: 0s - loss: 8.0458 - accuracy: 0.0999
Epoch 2: val_accuracy improved from 0.13110 to 0.14747, saving model to weights-improvement-02-0.147475.hdf5
2646/2646 [==============================] - 49s 18ms/step - loss: 8.0454 - accuracy: 0.1000 - val_loss: 6.4401 - val_accuracy: 0.1475
Epoch 3/150
2641/2646 [============================>.] - ETA: 0s - loss: 7.9681 - accuracy: 0.1213
Epoch 3: val_accuracy did not improve from 0.14747
2646/2646 [==============================] - 15s 6ms/step - loss: 7.9707 - accuracy: 0.1212 - val_loss: 9.6840 - val_accuracy: 0.0934
Epoch 4/150
2641/2646 [============================>.] - ETA: 0s - loss: 7.9546 - accurac